## Įvadas į regresiją - Pamoka 1

#### Įdėkime į kontekstą

✅ Yra daugybė regresijos metodų, ir kurį pasirinkti priklauso nuo klausimo, į kurį norite atsakyti. Jei norite prognozuoti tikėtiną žmogaus ūgį pagal jo amžių, naudotumėte `linijinę regresiją`, nes ieškote **skaitinės vertės**. Jei jus domina, ar tam tikra virtuvė turėtų būti laikoma veganiška, ieškote **kategorijos priskyrimo**, todėl naudotumėte `logistinę regresiją`. Apie logistinę regresiją sužinosite vėliau. Pagalvokite apie keletą klausimų, kuriuos galite užduoti duomenims, ir kuris iš šių metodų būtų tinkamesnis.

Šioje dalyje dirbsite su [mažu duomenų rinkiniu apie diabetą](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Įsivaizduokite, kad norėtumėte išbandyti gydymą diabetu sergantiems pacientams. Mašininio mokymosi modeliai galėtų padėti nustatyti, kurie pacientai geriau reaguotų į gydymą, remiantis kintamųjų deriniais. Net ir labai paprastas regresijos modelis, vizualizuotas, galėtų parodyti informaciją apie kintamuosius, kurie padėtų jums organizuoti teorinius klinikinius tyrimus.

Taigi, pradėkime šią užduotį!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Piešinys @allison_horst</figcaption>

<!--![Piešinys \@allison_horst](../../../../../../translated_images/encouRage.e75d5fe0367fb9136b78104baf4e2032a7622bc42a2bc34c0ad36c294eeb83f5.lt.jpg)<br>Piešinys @allison_horst-->


## 1. Įrankių rinkinio paruošimas

Šiai užduočiai mums reikės šių paketų:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) yra [R paketų rinkinys](https://www.tidyverse.org/packages), sukurtas tam, kad duomenų mokslas būtų greitesnis, paprastesnis ir įdomesnis!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) sistema yra [paketų rinkinys](https://www.tidymodels.org/packages/) modeliavimui ir mašininio mokymosi užduotims.

Juos galite įdiegti naudodami šią komandą:

`install.packages(c("tidyverse", "tidymodels"))`

Žemiau pateiktas skriptas patikrina, ar turite visus reikalingus paketus šiam moduliui atlikti, ir, jei kai kurių trūksta, juos įdiegia.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Dabar įkelkime šiuos nuostabius paketus ir padarykime juos prieinamus dabartinėje R sesijoje. (Tai tik iliustracija, `pacman::p_load()` jau tai padarė už jus)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. Diabeto duomenų rinkinys

Šiame pratime pademonstruosime savo regresijos įgūdžius, prognozuodami diabeto duomenų rinkinio pagrindu. [Diabeto duomenų rinkinys](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) apima `442 pavyzdžius` duomenų apie diabetą, su 10 prognozuojamųjų kintamųjų: `amžius`, `lytis`, `kūno masės indeksas`, `vidutinis kraujo spaudimas` ir `šeši kraujo serumo matavimai`, taip pat rezultato kintamąjį `y`: kiekybinį ligos progresavimo matą praėjus vieneriems metams po pradinio taško.

|Stebėjimų skaičius|442|
|------------------|:---|
|Prognozuojamųjų kintamųjų skaičius|Pirmos 10 skiltelių yra skaitiniai prognozuojamieji|
|Rezultatas/Tikslas|11-oji skiltis yra kiekybinis ligos progresavimo matas praėjus vieneriems metams po pradinio taško|
|Prognozuojamųjų kintamųjų informacija|- amžius metais
||- lytis
||- kūno masės indeksas (bmi)
||- vidutinis kraujo spaudimas (bp)
||- s1 tc, bendras serumo cholesterolis
||- s2 ldl, mažo tankio lipoproteinai
||- s3 hdl, didelio tankio lipoproteinai
||- s4 tch, bendras cholesterolis / HDL
||- s5 ltg, galbūt serumo trigliceridų lygio logaritmas
||- s6 glu, kraujo cukraus lygis|

> 🎓 Atminkite, tai yra prižiūrimas mokymasis, todėl mums reikia pavadinto 'y' tikslo.

Prieš manipuliuodami duomenimis su R, turite importuoti duomenis į R atmintį arba sukurti ryšį su duomenimis, kad R galėtų pasiekti duomenis nuotoliniu būdu.

> [readr](https://readr.tidyverse.org/) paketas, kuris yra Tidyverse dalis, suteikia greitą ir patogų būdą skaityti stačiakampius duomenis į R.

Dabar įkelsime diabeto duomenų rinkinį iš šio šaltinio URL: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Taip pat atliksime duomenų patikrinimą naudodami `glimpse()` ir parodysime pirmas 5 eilutes naudodami `slice()`.

Prieš tęsdami, pristatykime dar vieną dalyką, kurį dažnai sutiksite R kode 🥁🥁: vamzdžio operatorių `%>%`

Vamzdžio operatorius (`%>%`) atlieka operacijas logine seka, perduodamas objektą į priekį į funkciją arba iškvietimo išraišką. Galite galvoti apie vamzdžio operatorių kaip sakantį "ir tada" jūsų kode.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` parodo, kad šie duomenys turi 442 eilutes ir 11 stulpelių, kurių visi stulpeliai yra duomenų tipo `double`.

<br>

> `glimpse()` ir `slice()` yra funkcijos [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, dalis Tidyverse, yra duomenų manipuliavimo gramatika, kuri suteikia nuoseklų veiksmų rinkinį, padedantį spręsti dažniausiai pasitaikančius duomenų manipuliavimo iššūkius.

<br>

Dabar, kai turime duomenis, susiaurinkime dėmesį į vieną savybę (`bmi`), kurią naudosime šiam pratimu. Tam reikės pasirinkti norimus stulpelius. Taigi, kaip tai padaryti?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) leidžia mums *pasirinkti* (ir, jei reikia, pervadinti) stulpelius duomenų rėmelyje.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Mokymo ir testavimo duomenys

Supervizuoto mokymosi praktikoje įprasta *padalyti* duomenis į du poskyrius: (paprastai didesnį) rinkinį, skirtą modelio mokymui, ir mažesnį „atsarginį“ rinkinį, skirtą patikrinti, kaip modelis veikia.

Dabar, kai turime paruoštus duomenis, galime patikrinti, ar mašina gali padėti nustatyti logišką skirstymą tarp skaičių šiame duomenų rinkinyje. Galime naudoti [rsample](https://tidymodels.github.io/rsample/) paketą, kuris yra Tidymodels sistemos dalis, kad sukurtume objektą, kuriame būtų informacija apie *kaip* padalyti duomenis, o tada dar dvi rsample funkcijas, kad išgautume sukurtus mokymo ir testavimo rinkinius:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Treniruokite linijinės regresijos modelį su Tidymodels

Dabar esame pasiruošę treniruoti savo modelį!

Tidymodels sistemoje modeliai apibrėžiami naudojant `parsnip()` ir nurodant tris pagrindinius aspektus:

-   Modelio **tipas** skiria modelius, tokius kaip linijinė regresija, logistinė regresija, sprendimų medžių modeliai ir panašiai.

-   Modelio **režimas** apima dažniausiai naudojamas parinktis, tokias kaip regresija ir klasifikacija; kai kurie modelių tipai palaiko abu režimus, o kai kurie turi tik vieną.

-   Modelio **variklis** yra skaičiavimo įrankis, kuris bus naudojamas modelio pritaikymui. Dažnai tai yra R paketai, tokie kaip **`"lm"`** arba **`"ranger"`**

Ši modelio informacija yra užfiksuota modelio specifikacijoje, tad sukurkime ją!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Kai modelis yra *nustatytas*, jį galima `įvertinti` arba `treniruoti` naudojant [`fit()`](https://parsnip.tidymodels.org/reference/fit.html) funkciją, paprastai naudojant formulę ir tam tikrus duomenis.

`y ~ .` reiškia, kad mes pritaikysime `y` kaip prognozuojamą dydį/taikinį, paaiškinamą visais prediktoriais/savybėmis, t. y. `.` (šiuo atveju turime tik vieną prediktorių: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

Iš modelio rezultatų matome koeficientus, išmoktus mokymo metu. Jie atspindi geriausiai tinkančios linijos koeficientus, kurie suteikia mažiausią bendrą klaidą tarp faktinės ir numatytos kintamojo reikšmės.
<br>

## 5. Prognozių sudarymas testavimo rinkiniui

Dabar, kai apmokėme modelį, galime jį naudoti ligos progresavimo y prognozėms testavimo duomenų rinkiniui, pasitelkdami [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Tai bus naudojama linijai tarp duomenų grupių nubrėžti.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Oho! 💃🕺 Mes ką tik apmokėme modelį ir panaudojome jį prognozėms!

Atliekant prognozes, tidymodels konvencija visada yra pateikti tibble/duomenų rėmelį su standartizuotais stulpelių pavadinimais. Tai leidžia lengvai sujungti pradinius duomenis ir prognozes į patogų formatą tolimesnėms operacijoms, tokioms kaip vizualizavimas.

`dplyr::bind_cols()` efektyviai sujungia kelis duomenų rėmelius stulpeliais.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Modeliavimo rezultatų vizualizavimas

Dabar metas tai pamatyti vizualiai 📈. Sukursime sklaidos diagramą, kurioje bus pateiktos visos testų rinkinio `y` ir `bmi` reikšmės, o tada naudosime prognozes, kad nubrėžtume liniją tinkamiausioje vietoje, tarp modelio duomenų grupių.

R turi kelias sistemas grafikai kurti, tačiau `ggplot2` yra viena elegantiškiausių ir universaliausių. Ji leidžia sudaryti grafikus **derinant nepriklausomus komponentus**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Pagalvokite, kas čia vyksta. Tiesi linija eina per daugybę mažų duomenų taškų, bet ką ji iš tikrųjų daro? Ar galite suprasti, kaip ši linija turėtų padėti numatyti, kur naujas, nematytas duomenų taškas turėtų atsidurti santykyje su grafiko y ašimi? Pabandykite žodžiais apibūdinti praktinį šio modelio panaudojimą.

Sveikiname, jūs sukūrėte savo pirmąjį linijinės regresijos modelį, atlikote prognozę su juo ir pavaizdavote ją grafike!



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama profesionali žmogaus vertimo paslauga. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius naudojant šį vertimą.
